## Advent of code 2022 alternative solutions for days 19 and 22
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2022 day 22 alternative solution using simpler warping; there is no rotation, instead
# both for part 1 and part 2 warps are used as soon as leaving the current face, where
# exit and entry positions along the relevant edges/axes are determined

sample2='''
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
'''

sample2_faces=[[0, 0, 1, 0], [2, 3, 4, 0], [0, 0, 5, 6]] # ids of faces on map
sample2_facesz=4 # height and width of each face
sample2_warps_part1={ # per face lists the adjacent faces to right, down, left, up and new movement direction (versus the flat map)
    1: [(1, 0), (4, 1), (1, 2), (5, 3)],
    2: [(3, 0), (2, 1), (4, 2), (2, 3)],
    3: [(4, 0), (3, 1), (2, 2), (3, 3)],
    4: [(2, 0), (5, 1), (3, 2), (1, 3)],
    5: [(6, 0), (1, 1), (6, 2), (4, 3)],
    6: [(5, 0), (6, 1), (5, 2), (6, 3)]
}
sample2_warps_part2={ 
    1: [(6, 2), (4, 1), (3, 1), (2, 1)],
    2: [(3, 0), (5, 3), (6, 3), (1, 1)],
    3: [(4, 0), (5, 0), (2, 2), (1, 0)],
    4: [(6, 1), (5, 1), (3, 2), (1, 3)],
    5: [(6, 0), (2, 3), (3, 3), (4, 3)],
    6: [(1, 2), (2, 0), (5, 2), (4, 2)]
}
sample2_data=dict(faces=sample2_faces, sz=sample2_facesz, warps1=sample2_warps_part1, warps2=sample2_warps_part2)

sample1_faces=[[0, 5, 6], [0, 4, 0], [2, 3, 0], [1, 0, 0]]
sample1_facesz=50
sample1_warps_part1={ 
    1: [(1, 0), (2, 1), (1, 2), (2, 3)],
    2: [(3, 0), (1, 1), (3, 2), (1, 3)],
    3: [(2, 0), (5, 1), (2, 2), (4, 3)],
    4: [(4, 0), (3, 1), (4, 2), (5, 3)],
    5: [(6, 0), (4, 1), (6, 2), (3, 3)],
    6: [(5, 0), (6, 1), (5, 2), (6, 3)]
}
sample1_warps_part2={ 
    1: [(3, 3), (6, 1), (5, 1), (2, 3)],
    2: [(3, 0), (1, 1), (5, 0), (4, 0)],
    3: [(6, 2), (1, 2), (2, 2), (4, 3)],
    4: [(6, 3), (3, 1), (2, 1), (5, 3)],
    5: [(6, 0), (4, 1), (2, 0), (1, 0)],
    6: [(3, 2), (4, 2), (5, 2), (1, 3)]
}
sample1_data=dict(faces=sample1_faces, sz=sample1_facesz, warps1=sample1_warps_part1, warps2=sample1_warps_part2)

def do_step(x, y, direc, faces, sz, warps):
    '''from x,y take a single step in direction direc, return new position, if moving outside
    current face then warp instead'''
    # find current face
    fx0=x // sz
    fy0=y // sz
    curface=faces[fy0][fx0]
    # normal movement
    nx=x
    ny=y
    if direc==0:
        nx+=1
    elif direc==1:
        ny+=1
    elif direc==2:
        nx-=1
    elif direc==3:
        ny-=1
    else:
        assert False
    # warp check
    if nx//sz==fx0 and ny//sz==fy0: # same face so same direction
        return nx,ny,direc
    # find position along exiting edge, counting from the left, looking in the exiting direction
    along_mx=x % sz
    along_my=y % sz
    if direc==0:
        along=along_my
    elif direc==1:
        along=sz-1-along_mx
    elif direc==2:
        along=sz-1-along_my
    else:
        assert direc==3
        along=along_mx
    # new face, direction, topleft of new face
    newface,ndirec=warps[curface][direc]
    facefound=False
    for fy1 in range(len(faces)):
        for fx1 in range(len(faces[0])):
            if faces[fy1][fx1]==newface:
                facefound=True
                break
        if facefound:
            break
    nx=fx1*sz
    ny=fy1*sz
    # entry point using nx,ny and along and ndirec
    if ndirec==0:
        ny+=along
    elif ndirec==1:
        nx+=sz-1-along
    elif ndirec==2:
        nx+=sz-1
        ny+=sz-1-along
    else:
        assert ndirec==3
        ny+=sz-1
        nx+=along
    return nx,ny,ndirec

# do_warp tests for sample2
assert do_step(10, 6, 0,  sample2_data['faces'], sample2_data['sz'], sample2_data['warps1'])==(11, 6, 0)
assert do_step(11, 6, 0,  sample2_data['faces'], sample2_data['sz'], sample2_data['warps1'])==(0, 6, 0)
assert do_step(5, 7, 1,   sample2_data['faces'], sample2_data['sz'], sample2_data['warps1'])==(5, 4, 1)

assert do_step(11, 5, 0,  sample2_data['faces'], sample2_data['sz'], sample2_data['warps2'])==(14, 8, 1)
assert do_step(10, 11, 1, sample2_data['faces'], sample2_data['sz'], sample2_data['warps2'])==(1, 7, 3)

def do_walk1(board, walk, x, y, direc, faces, sz, warps):
    assert board[y][x]=='.'
    for w in walk:
        if w=='L':
            direc=(direc-1)%4
        elif w=='R':
            direc=(direc+1)%4
        else:
            assert isinstance(w, int)
            for _ in range(w):
                nx,ny,ndirec=do_step(x, y, direc, faces, sz, warps)
                if board[ny][nx]=='#':
                    break
                x=nx
                y=ny
                direc=ndirec
                assert board[y][x]=='.'
    return x,y,direc

def do_walk(board, walk, sample_data, do_part):
    # find start
    y=0 # coords are 0-based
    x=board[0].index('.')
    direc=0 # right=0, 1=down, 2=left, 3=up
    print(f'start: {x=}, {y=}, {direc=}')
    # execute walk
    warps=sample_data['warps1'] if do_part==1 else sample_data['warps2']
    endpos=do_walk1(board, walk, x, y, direc, sample_data['faces'], sample_data['sz'], warps)
    # passwd
    x,y,direc=endpos
    print(f'end: {x=}, {y=}, {direc=}')
    res=1000*(y+1)+4*(x+1)+direc
    return res

sample1=open('data_src/2022-day-22-input.txt').read()
sample=sample1 # use this line to switch between sample1 and sample2
sample_data=sample2_data if sample==sample2 else sample1_data
groups=zio.get_line_groups(sample.splitlines(), nostrip=True)
assert len(groups)==2
assert len(groups[1])==1
board=groups[0]
walk=re.split(r'([RL])', groups[1][0])
walk=[(s if s in {'L', 'R'} else int(s)) for s in walk]

# part 1
passwd=do_walk(board, walk, sample_data, do_part=1)
print(f'part 1: {passwd}')

# part 2
passwd=do_walk(board, walk, sample_data, do_part=2)
print(f'part 2: {passwd}')

# (for sample2 the answers are 6032 and 5031)
# part 1: 136054
# part 2: 122153

In [ ]:
# 2022 day 19 alternative solution using BFS with suboptimal solutions
# and sorting of the todo list

sample2='''
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
'''

Blueprint=collections.namedtuple('Blueprint', \
    ['id', 'ore_rob_cost', 'clay_rob_cost', 'obs_rob_ore_cost', 'obs_rob_clay_cost', 'geo_rob_ore_cost',\
     'geo_rob_obs_cost'])

State=collections.namedtuple('State', \
    ['t', 'ore_robs', 'ore', 'clay_robs', 'clay', 'obs_robs', 'obs', 'geo_robs', 'geos'])

class StateM:
    def __init__(self, s=None):
        self.t=s.t if s else 0
        self.ore_robs=s.ore_robs if s else 0
        self.ore=s.ore if s else 0
        self.clay_robs=s.clay_robs if s else 0
        self.clay=s.clay if s else 0
        self.obs_robs=s.obs_robs if s else 0
        self.obs=s.obs if s else 0
        self.geo_robs=s.geo_robs if s else 0
        self.geos=s.geos if s else 0
    
    def as_tuple(self):
        return State(self.t, self.ore_robs, self.ore, self.clay_robs, self.clay, self.obs_robs, self.obs,
         self.geo_robs, self.geos)

def get_step_states(state, bp):
    '''given state, a State namedtuple generate a list of states reachable after one step,
    again as State namedtuples'''
    res=[]
    assert isinstance(state, State)
    s=StateM(state)
    old_ore=s.ore
    old_clay=s.clay
    old_obs=s.obs
    old_geos=s.geos
    # collections at end of minute can be done now already
    s.ore+=s.ore_robs
    s.clay+=s.clay_robs
    s.obs+=s.obs_robs
    s.geos+=s.geo_robs
    s.t+=1
    # try building any of the robots, first geo
    build_failed=False
    if old_ore>=bp.geo_rob_ore_cost and old_obs>=bp.geo_rob_obs_cost:
        s.ore-=bp.geo_rob_ore_cost
        s.obs-=bp.geo_rob_obs_cost
        s.geo_robs+=1
        res.append(s.as_tuple())
        s.ore+=bp.geo_rob_ore_cost
        s.obs+=bp.geo_rob_obs_cost
        s.geo_robs-=1
    else:
        build_failed=True
    if old_ore>=bp.obs_rob_ore_cost and old_clay>=bp.obs_rob_clay_cost:
        s.ore-=bp.obs_rob_ore_cost
        s.clay-=bp.obs_rob_clay_cost
        s.obs_robs+=1
        res.append(s.as_tuple())
        s.ore+=bp.obs_rob_ore_cost
        s.clay+=bp.obs_rob_clay_cost
        s.obs_robs-=1
    else:
        build_failed=True
    if old_ore>=bp.clay_rob_cost:
        s.ore-=bp.clay_rob_cost
        s.clay_robs+=1
        res.append(s.as_tuple())
        s.ore+=bp.clay_rob_cost
        s.clay_robs-=1
    else:
        build_failed=True
    if old_ore>=bp.ore_rob_cost:
        s.ore-=bp.ore_rob_cost
        s.ore_robs+=1
        res.append(s.as_tuple())
        s.ore+=bp.ore_rob_cost
        s.ore_robs-=1
    else:
        build_failed=True
    if build_failed:
        # the no action option if any of the others failed
        res.append(s.as_tuple())
    return res

def open_geos_bfs(bp, time_limit):
    max_geodeficit=2 # how suboptimal is state allowed to be?
    max_oresurplus=10+max(bp.geo_rob_ore_cost, bp.obs_rob_ore_cost) # how many superfluous ore are we allowed?
    todos=sortedcontainers.SortedSet(key=lambda s: -s.geos) # each todo is a State named tuple
    startpos=StateM()
    startpos.ore_robs=1
    startpos=startpos.as_tuple()
    todos.add(startpos)
    explored=set() # all todos collected
    explored.add(startpos)
    reached={} # maps turn to (ore, geos)
    reached[0]=(0,0)
    while len(todos)>0:
        todo=todos.pop(0)
        if todo.t>=time_limit:
            continue
        newstates=get_step_states(todo, bp)
        for newstate in newstates:
            if newstate.t not in reached or reached[newstate.t][1]<newstate.geos:
                reached[newstate.t]=(newstate.ore, newstate.geos)
            if newstate.geos>=reached[newstate.t][1]-max_geodeficit and \
             newstate.ore<=reached[newstate.t][0]+max_oresurplus and newstate not in explored:
                todos.add(newstate)
                explored.add(newstate)
    return max([tup[1] for tup in reached.values()])

sample1=open('data_src/2022-day-19-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'Blueprint (\d+): Each ore robot costs (\d+) ore.\s*Each clay robot costs (\d+) ore.\s*Each obsidian robot costs (\d+) ore and (\d+) clay.\s*Each geode robot costs (\d+) ore and (\d+) obsidian.', s)) ]
data=[ [int(s) for s in row] for row in data]
# part 1
score=0
for bluep in data:
    bp=Blueprint(*bluep)
    geos=open_geos_bfs(bp, 24)
    print(f'bp {bp.id}: {geos} geos')
    score+=geos*bp.id
print(f'part 1: {score}')

# part 2
score=1
for bluep in data[:3]:
    bp=Blueprint(*bluep)
    geos=open_geos_bfs(bp, 32)
    print(f'bp {bp.id}: {geos} geos')
    score*=geos
print(f'part 2: {score}')

#bp 1: 0 geos
#bp 2: 4 geos
#bp 3: 0 geos
#bp 4: 0 geos
#bp 5: 0 geos
#bp 6: 8 geos
#bp 7: 0 geos
#bp 8: 0 geos
#bp 9: 0 geos
#bp 10: 3 geos
#bp 11: 0 geos
#bp 12: 1 geos
#bp 13: 1 geos
#bp 14: 3 geos
#bp 15: 2 geos
#bp 16: 1 geos
#bp 17: 1 geos
#bp 18: 1 geos
#bp 19: 0 geos
#bp 20: 0 geos
#bp 21: 1 geos
#bp 22: 0 geos
#bp 23: 4 geos
#bp 24: 2 geos
#bp 25: 7 geos
#bp 26: 0 geos
#bp 27: 0 geos
#bp 28: 0 geos
#bp 29: 0 geos
#bp 30: 1 geos
# part 1: 600 in 3 min.

#bp 1: 10 geos
#bp 2: 40 geos
#bp 3: 15 geos
#part 2: 6000 in 1 min.

In [ ]:
# TEMPLATE
# 2022 day 21
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-21-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed